In [43]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [37]:
df = pd.read_csv('data_for_student_case.csv')
df.shape
df

,txid,bookingdate,issuercountrycode,txvariantcode,bin,amount,currencycode,shoppercountrycode,shopperinteraction,simple_journal,cardverificationcodesupplied,cvcresponsecode,creationdate,accountcode,mail_id,ip_id,card_id
0,1,2015-11-09 14:26:51,MX,mccredit,530056.0,64800.0,MXN,MX,Ecommerce,Chargeback,True,0,2015-07-01 23:03:11,MexicoAccount,email68370,ip111778,card184798
1,2,2015-11-09 14:27:38,MX,mccredit,547046.0,44900.0,MXN,MX,Ecommerce,Chargeback,True,0,2015-07-02 04:50:55,MexicoAccount,email101299,ip78749,card151595
2,3,2015-11-23 16:34:16,MX,mccredit,528843.0,149900.0,MXN,MX,Ecommerce,Chargeback,True,0,2015-07-02 14:30:28,MexicoAccount,email278604,ip70594,card242142
3,4,2015-11-23 16:34:51,MX,mccredit,547146.0,109900.0,MXN,MX,Ecommerce,Chargeback,True,0,2015-07-03 07:53:37,MexicoAccount,email47409,ip113648,card181744
4,5,2015-11-09 14:26:08,MX,visaclassic,477291.0,89900.0,MXN,MX,Ecommerce,Chargeback,True,0,2015-07-08 18:35:35,MexicoAccount,email205501,ip83553,card97271
5,6,2015-11-09 14:26:25,MX,visaclassic,493161.0,79900.0,MXN,MX,Ecommerce,Chargeback,True,0,2015-07-08 23:04:26,MexicoAccount,email243511,ip112410,card159816
6,7,2015-11-09 14:26:47,MX,visaclassic,477291.0,69900.0,MXN,MX,Ecommerce,Chargeback,True,0,2015-07-09 02:31:36,MexicoAccount,email137086,ip83481,card251249
7,8,2015-11-23 16:34:29,MX,mccredit,520416.0,129900.0,MXN,MX,Ecommerce,Chargeback,True,0,2015-07-09 04:56:48,MexicoAccount,email295644,ip66704,card70297
8,9,2015-12-08 19:58:35,MX,visaclassic,477291.0,129900.0,MXN,MX,Ecommerce,Chargeback,True,0,2015-07-09 22:55:40,MexicoAccount,email115605,ip68088,card39791
9,10,2015-11-09 14:25:50,MX,mccredit,549138.0,99900.0,MXN,MX,Ecommerce,Chargeback,True,0,2015-07-11 09:44:14,MexicoAccount,email182966,ip113025,card160922


In [38]:
# Remove confused transaction 
df = df[df['simple_journal'] != 'Refused']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 237036 entries, 0 to 290381
Data columns (total 17 columns):
txid                            237036 non-null int64
bookingdate                     237036 non-null object
issuercountrycode               237032 non-null object
txvariantcode                   237036 non-null object
bin                             237036 non-null float64
amount                          237036 non-null float64
currencycode                    237036 non-null object
shoppercountrycode              236694 non-null object
shopperinteraction              237036 non-null object
simple_journal                  237036 non-null object
cardverificationcodesupplied    223842 non-null object
cvcresponsecode                 237036 non-null int64
creationdate                    237036 non-null object
accountcode                     237036 non-null object
mail_id                         237036 non-null object
ip_id                           237036 non-null object
card_id  

In [33]:
# Convert bookingdate and creation date (string) into datetime 
cols = ['bookingdate', 'creationdate']
for col in cols:
    pd.to_datetime(df[col], format='%Y-%m-%d %H:%M:%S', errors='coerce')

count                  237036
unique                 232824
top       2015-08-22 10:12:01
freq                        6
Name: creationdate, dtype: object

Single variable exploration

In [ ]:
# Visualize money attribute
benign_data = df.loc[df['simple_journal'] == 'Settled']
frad_data = df.loc[df['simple_journal'] == 'Chargeback']

f, axarr = plt.subplots(2)
kwargs = dict(alpha=0.5, bins=100)
axarr[0].hist(benign_data['amount'], **kwargs, color='r', label='benign')
axarr[0].legend()
axarr[1].hist(frad_data['amount'], **kwargs, color='g', label='frad')
axarr[1].legend()
f.text(0.5, 0.01, 'Money', ha='center', va='center')
f.text(0.06, 0.5, 'Count', ha='center', va='center', rotation='vertical')
axarr[0].set_title("Histogram of Money")
plt.show()